In [96]:
#Packages used

#pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org pandas as pd -vvv
#pip install openpyxl
#python -m  py install dash
#pip install dash-bootstrap-components
#pip install numpy

In [97]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, callback, dash_table, State
from datetime import datetime as dt
from datetime import timedelta
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import random
import plotly.express as px
from plotly.subplots import make_subplots




In [98]:
test_data_new = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'TEST') # Import data from new database
test_data_new.dropna(how='all')
test_data_new.dropna(axis=1, how='all', inplace=True)

test_data_old= pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'OLD') # Import data from old database
test_data_old.dropna(how='all')

test_data_old.dropna(axis=1, how='all', inplace=True)  

total_data = pd.concat([test_data_old, test_data_new], ignore_index=True) # Concatenate databases

total_data.head(5)

c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed

c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed



,FAMILY,P_N,POWER.W,PRODUCTION.START,PRODUCTION.P,PRODUCTION.STATUS,CELL,FLUIDO.CHILLER,T.FLUIDO,T.AMBIENTE,...,STIMATION,OPERATORE,START.DATE,DURATION,TEST.TYPE,CELL.CHECK,TEST.REMARKS,P.N_CHECK,DECLARED.POWER.W,CIRCUITS
0,TAL,TAL29EMSBC00000,2900,COMPLETED,100.0,READY,Chiller,W,17.6,32.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TLA,TLA57EMSBC00000,6000,COMPLETED,100.0,READY,Chiller,W,18.1,32.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TAL,TAL93EMSBC00000,9550,COMPLETED,100.0,READY,Chiller,W,19.6,32.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TAL,TAL93EMSBC00000,9550,COMPLETED,100.0,READY,Chiller,W,19.9,32.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TAL,TAL76EMSBC00000,8100,COMPLETED,100.0,READY,Chiller,W,20.0,32.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# CALENDAR FROM CELLS AND OPERATORS

cells_calendar = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'CELLS') # Import data from new database
#cells_calendar.head()

operatos_calendar = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'OPERATORS') # Import data from new database
#operators_calendar.head()

In [100]:
#DECLARE FLUID PARAMETERS

cp_water = 4186 # J.(KG.°C)^(-1)
specific_mass_water = 1000 #kg/m3

cp_oil = 1880 # J.(KG.°C)^(-1)
specific_mass_oil = 900 #kg/m3

cp_glycol = 2533 # J.(KG.°C)^(-1)
specific_mass_glycol = 1040 #kg/m3


In [101]:
#CALCULATE CP, RO PARAMETERS

total_data['PERC.M']= pd.to_numeric(total_data['PERC.M'], errors='coerce')
convertion_flow = 0.06 # convert flow from l/min to m3/s

total_data['CP']= 0 # Declare specific heat J.(KG.°C)^(-1)
total_data['RO']= 0 # Declare specific heat J.(KG.°C)^(-1)

total_data["RO"] = np.where(total_data["FLUIDO.CHILLER"] == 'W', specific_mass_water, 
                            (np.where(total_data["FLUIDO.CHILLER"] == 'O', specific_mass_oil, 
                                      (np.where(total_data["FLUIDO.CHILLER"] == 'E', (((100-total_data["PERC.M"])*specific_mass_water) + (total_data["PERC.M"]*specific_mass_oil))/100,
                                                (np.where(total_data["FLUIDO.CHILLER"] == 'G', (((100-total_data["PERC.M"])*specific_mass_water) + (total_data["PERC.M"]*specific_mass_glycol))/100, 0          
                                      )))))))

total_data["CP"] = np.where(total_data["FLUIDO.CHILLER"] == 'W', cp_water, 
                            (np.where(total_data["FLUIDO.CHILLER"] == 'O', cp_oil, 
                                      (np.where(total_data["FLUIDO.CHILLER"] == 'E', (((100-total_data["PERC.M"])*cp_water) + (total_data["PERC.M"]*cp_oil))/100,
                                                (np.where(total_data["FLUIDO.CHILLER"] == 'G', (((100-total_data["PERC.M"])*cp_water) + (total_data["PERC.M"]*cp_glycol))/100, 0          
                                      )))))))  

#total_data[['FLUIDO.CHILLER','WORK.FLUID','CP','RO', 'PERC.M']].head()


In [102]:
# POWER CALCULATION

convertion_flow = 0.00001666 # convert flow from l/min to m3/s

total_data['DELTA'] = (total_data['T.OUT.F.M_C']-total_data['T.IN.F.M_C']).abs() # [°C]

total_data['CALCULATE.POWER'] = total_data['FLUID.FLOW.L_MIN']*convertion_flow * total_data['CP'] * total_data['DELTA'] # [kW]

#total_data[['P_N','FLUID.FLOW.L_MIN', 'CP', 'DELTA', 'CALCULATE.POWER']].head()

In [103]:
# SETTIME PARAMETERS AS DATETIME

total_data['STIMATION'] =  pd.to_timedelta(total_data['STIMATION'], unit='h')
total_data['DURATION'] =  pd.to_timedelta(total_data['DURATION'], unit='h')

#total_data[['AVAILABILITY', 'DUE.DATE', 'STIMATION', 'START.DATE', 'END.DATE', 'DURATION']].dtypes

In [104]:
# TIME CALCULATION

#SET INTEREST INTERVAL

start_date = dt(2024, 9, 10) # (Y/M/D/H/min...)
end_date = dt(2024, 10, 10) # (Y/M/D/H/min...)

#____________________________________________________________________________________________________________________________________
#EXTRACT BUSSINESS DAYS BETWEEN THE DATES

work_days = pd.bdate_range(start_date, end_date) #Defines number of bussiness days between stard date and end date
#print(len(work_days))

#_____________________________________________________________________________________________________________________________________
#FILTER DATAFRAME BY DATE
mask = (total_data['AVAILABILITY']  >= start_date)  & (total_data['DUE.DATE'] <= end_date)

total_data_filtered_by_date = total_data.loc[mask] # all Data filtered by set date

cell_usage_table = total_data_filtered_by_date[['CELL', 'STIMATION', 'DURATION']] #Time table

#_____________________________________________________________________________________________________________________________________
#CALCULATE TOTAL TIME FOR EACH CELL

cell_hours_usage = cell_usage_table.groupby(['CELL']).sum().reset_index() #Total time grouped by cell

#display(cell_hours_usage)

#_____________________________________________________________________________________________________________________________________
#CELL PREDICTED AND EFFETIVE USAGE

cell_40_daily = timedelta(hours = 10) # Max usage of the cell
cell_60_daily = timedelta(hours=6) # Max usage of the cell
cell_110_daily = timedelta(hours=8) # Max usage of the cell

cell_40_availability = len(work_days)*cell_40_daily #availability in the set period
cell_60_availability = len(work_days)*cell_60_daily #availability in the set period
cell_110_availability = len(work_days)*cell_110_daily #availability in the set period

cell_hours_usage['CELL.AVAILABILITY'] = np.where(cell_hours_usage['CELL'] == 40, cell_40_availability,
                                                 (np.where(cell_hours_usage['CELL'] == 60, cell_60_availability,
                                                           np.where(cell_hours_usage['CELL'] == 110, cell_110_availability,0)   
                                                 )))

cell_hours_usage['STIMATED.USAGE'] = cell_hours_usage['STIMATION']*100/cell_hours_usage['CELL.AVAILABILITY']
cell_hours_usage['EFFETIVE.USAGE'] = cell_hours_usage['DURATION']*100/cell_hours_usage['CELL.AVAILABILITY']
                               
display(cell_hours_usage)

,CELL,STIMATION,DURATION,CELL.AVAILABILITY,STIMATED.USAGE,EFFETIVE.USAGE
0,60,0 days 08:00:00,1 days 10:00:00,5 days 18:00:00,5.797101,24.637681
1,110,1 days 00:00:00,2 days 16:48:00,7 days 16:00:00,13.043478,35.217391


In [105]:
# PLOTTING STIMATED USE CELL 060

if 60 in cell_hours_usage['CELL'].values:

    fig1 = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = cell_hours_usage.loc[cell_hours_usage['CELL'] == 60]['STIMATED.USAGE'][0],
    mode = "gauge+number",
    title = {'text': "CURRENT USAGE - STIMATED - CELL 060"},
   
    gauge = { 'axis': {'range': [None, 100]},
             'steps' : [
                 {'range': [0, 50], 'color': "lightgray"},
                 {'range': [50, 75], 'color': "gray"}],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 95}}))
    fig1.show()
    
else:
    fig2 = go.Figure(go.Indicator())
    fig2.show()

In [106]:
# EFETIVE USAGE CELL 60

if 60 in cell_hours_usage['CELL'].values:

    fig3 = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = cell_hours_usage.loc[cell_hours_usage['CELL'] == 60]['EFFETIVE.USAGE'][0],
    mode = "gauge+number",
    title = {'text': "EFETIVE USAGE - STIMATED - CELL 060"},
   
    gauge = { 'axis': {'range': [None, 100]},
             'steps' : [
                 {'range': [0, 50], 'color': "lightgray"},
                 {'range': [50, 75], 'color': "gray"}],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 95}}))
    fig3.show()
    
else:
    fig4 = go.Figure(go.Indicator())
    fig4.show()

In [107]:
#NAV BAR DASHBOARD

NVent_LOGO = "https://www.nvent.com/themes/custom/particle/dist/app-drupal/assets/images/logo-nvent.svg" # Address to NVent logo

row_content = dbc.Row(
                     [  
                        dbc.Col(" ", width="10"),
                        dbc.Col(html.Img(src=NVent_LOGO, height="40px")),
                        dbc.Col(dbc.NavbarBrand("TEXA LAB DASHBOARD") ),
                     ],
                     )

navbar = dbc.Navbar(

                     html.A( 
                            row_content, href="https://www.nvent.com/en-us/"                   
                           ),
                               color="dark",
                               dark=True,
                    )

In [108]:
# DEFINES THE DYNAMIC TABLE FOR FILTERING THE LAB DATA
status_table = total_data[['CELL', 'TEST.ID', "P_N", "DUE.DATE", "TEST.DESCRIPTION", "TEST.STATUS"]]
#status_table['DUE.DATE'] = status_table['DUE.DATE'].dt.strftime('%m/%d/%Y')

dynamic_table =  html.Div( 
                            dash_table.DataTable(
                                                    columns=[{"name": i, "id": i} for i in status_table.columns],
                                                    page_size=10,
                                                    fixed_rows={'headers': True},
                                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                                    id= "total_data2",
                                                    data=status_table.to_dict("records")                     
                                    )
                )
#display(status_table)

In [109]:
# MENU CELL

cell_option = dbc.Card(
                        [dbc.CardHeader("TEST CELL"),
                        dbc.CardBody(
                                    dcc.Checklist(
                                                options=[
                                                        {"label": "040", "value": 40},
                                                        {"label": "060", "value": 60},
                                                        {"label": "110", "value": 110},
                                                        {"label": "Chiller", "value": "Chiller", "disabled": True},
                                                        {"label": "Clima", "value": "Clima", "disabled": True}
                                                        ],
                                                        value=[110],
                                                        id = "selected_cell" )
                        )],
                        style={
                                'width': 120,
                                'height': 200,
                                'margin-left': 5,
                                'margin-right': 5,
                                'margin-top': 5,
                                'margin-bottom': 5
                                }                 
)
                     

In [115]:
status_option = dbc.Card(
                        [dbc.CardHeader("TEST CELL"),
                        dbc.CardBody(
                                    dcc.Checklist(
                                                options=[
                                                        {"label": "COMPLETED", "value": "COMPLETED", },
                                                        {"label": "ON GOING", "value": "ON GOING"},
                                                        {"label": "HOLDING", "value": "HOLDING"},
                                                        ],
                                                        value=["ON GOING"],
                                                        id = "status_cell"    
                                                                            )
                        )],
                        style={
                                'width': 180,
                                'height': 200,
                                'margin-left': 5,
                                'margin-right': 5,
                                'margin-top': 5,
                                'margin-bottom': 5
                                }                 
)

In [ ]:
, 
                                     html.Div(status_table, 
                                              style={
                                                    'width': 1000,
                                                    'height': 300,
                                                    'margin-left': 15,
                                                    'margin-top': 15,
                                                    'margin-bottom': 15
                                                    }
                                              )

In [119]:


app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])


app.layout = [ navbar, dbc.Container(         
                                    [html.Div(
                                                [
                                                    html.Div(
                                                                                style={
                                                                                        'width': 120,
                                                                                        'height': 100,
                                                                                        'margin-left': 5,
                                                                                        'margin-right': 5,
                                                                                        'margin-top': 5,
                                                                                        'margin-bottom': 5
                                                                                      }     
                                                            ),
                                                    html.Div(
                                                            dcc.Checklist(
                                                                                options=[
                                                                                        {"label": "040", "value": 40},
                                                                                        {"label": "060", "value": 60},
                                                                                        {"label": "110", "value": 110},
                                                                                        {"label": "Chiller", "value": "Chiller", "disabled": True},
                                                                                        {"label": "Clima", "value": "Clima", "disabled": True}
                                                                                        ],
                                                                                         value=[110],
                                                                                         id = "selected_cell" 
                                                                            )
                                                            ),         
                                                    html.Div(
                                                            dcc.Checklist(
                                                                                options=[
                                                                                        {"label": "COMPLETED", "value": "COMPLETED", },
                                                                                        {"label": "ON GOING", "value": "ON GOING"},
                                                                                        {"label": "HOLDING", "value": "HOLDING"},
                                                                                        ],
                                                                                         value=["ON GOING"],
                                                                                         id = "status_cell"    
                                                                            )
                                                            )
                                                ]
                                            ), 
                                                    html.Div(dynamic_table, style={
                                                                                    'width': 1000,
                                                                                    'height': 300,
                                                                                    'margin-left': 15,
                                                                                    'margin-top': 15,
                                                                                    'margin-bottom': 15
                                                                                    }
                                              ), 
                                                html.Div(
                                                    dcc.DatePickerRange(
                                                    id="date_filter",
                                                    start_date=total_data["DUE.DATE"].min(),
                                                    end_date=total_data["DUE.DATE"].max(),
                                                                        )
                                                )]
                                    ,

                                        fluid=True,
                                        style={'display': 'flex'},
                                        className='dashboard-container')    ]    

@app.callback(
    Output("dynamic_table", 'data'),
    Input('selected_cell', 'value'),
    Input('status_cell', 'value'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date")
 )
            

def update_table(selected_cell, status_cell, start_date, end_date):

    filtered_df= status_table[ 
                                status_table["DUE.DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date)) &
                                status_table['CELL'].isin(selected_cell)  &
                                status_table['TEST.STATUS'].isin(status_cell)
                             ] 

    return filtered_df.to_dict("records")


if __name__ == '__main__':
    app.run(debug=True)
    




#http://127.0.0.1:8050/